# Module 6 - Lab, part 2

# Introduction to Q-Learning 


In the previous lesson we explored Monte Carlo **reinforcement learning**. MC RL required that the returns for an entire episode be computed before any values are available for use. The disadvantage of this approach is the the full set of returns are required for state value or action value estimates. But, how can we get state values or action-values in fewer time steps? **Q-learning** is an off-policy time differencing algorithm, which is the focus of this lab. 

The interaction between a reinforcement learning agent and the environment are illustrated in the figure below. Notice that the only feedback the agent receives from the environment is reward and state.   

<img src="img/RL_AgentModel.JPG" alt="Drawing" style="width:500px; height:300px"/>
<center> **Reinforcement Learning Agent and Environment** </center>  

The ability to learn from experience is an attractive concept. This method of learning seems to mimic human learning. However, reinforcement learning has proven difficult to use in real-world applications. For a review of successes and problems arising when applying RL to robotics see [Kobler et. al.](https://www.ias.informatik.tu-darmstadt.de/uploads/Publications/Kober_IJRR_2013.pdf). At the present time, RL has mostly succeeded in cases where simulations can be used to gain experience. 


## On Policy vs. Off Policy Algorithms

In this lesson we will explore an example of a widely used **off policy** algorithm, Q-Learning. 

On policy methods evaluate and improve a single policy. On policy methods converge quickly and often to good solution. In general, **exploration** is performed using $\epsilon$-greedy methods. The TD(0) and MC algorithms we have examined are examples of on policy methods. On policy algorithms are known to have good convergence properties. 

In contrast, off policy methods use two policies. The policy the agent is following is called the **behavior policy**, denoted $b(A_t | S_t)$. The policy being improved is known as the **target policy**, denoted $\pi (A_t | S_t)$. The agent obtains samples of the environment while following the behavior policy. These samples are used to improve the target policy. An advantage of off policy methods is that a deterministic behavior policy can be used while a better target policy is developed. 

## Simulator for Grid World

As discussed in other labs, **Navigation** to a goal is a significant problem in robotics. Real-world navigation is rather complex. Therefore, in this example we will use a simple analog called a **grid world**. The grid world for this problem is shown below. 

<img src="img/GridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **A 4x4 Grid World with Terminal State** </center>

The grid world consists of a 4x4 set of positions the robot can occupy. Each position is considered a state. The goal is to navigate to state 0, the goal, in the minimum steps. We will explore methods to find policies which reach this goal and achieve maximum reward. 

Grid position 0 is the goal and a **terminal state**. There are no possible state transitions out of this position. The presence of a terminal state makes this an **episodic Markov random process**. For each episode sampled the robot can start in any other random position, $\{ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 \}$. This random selection process makes this a **random start** Q-learning algorithm. The episode terminates when the robot enters the terminal position (state 0).  

In reality, an RL agent may need to explore to find the possible actions when it is in some particular state. To simplify our example, we encode, or represent, these possibilities in a dictionary as shown in the code block below. We use a dictionary of dictionaries to perform the lookup. The keys of the outer dictionary are the identifiers (numbers) of the states. The keys of the inner dictionary are the possible actions and the values are the **successor state**, $s'$, for that transition.  

In each state, there are four possible actions the robot can take:
- up, u
- down, d,
- left, l
- right, r

The TD RL agent has no model for the environment. Therefore, beyond these allowed actions, all other information is encapsulated in the environment and is unobservable by the agent. This is the key difference between reinforcement learning and dynamic programming. 

Execute the code in the cell below to test the environment simulator. 

In [ ]:
## import numpy for latter
import numpy as np
import numpy.random as nr
import pandas as pd

## Define the transition dictonary of dictionaries:
neighbors = {0:{'u':0, 'd':0, 'l':0, 'r':0},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':9, 'd':13, 'l':12, 'r':14},
          14:{'u':10, 'd':14, 'l':13, 'r':15},
          15:{'u':11, 'd':15, 'l':14, 'r':15}}

rewards = {0:{'u':10.0, 'd':10.0, 'l':10.0, 'r':10.0},
          1:{'u':-1, 'd':-0.1, 'l':10.0, 'r':-0.1},
          2:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          4:{'u':10.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          5:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          8:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          10:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          11:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          12:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
          13:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          14:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          15:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}}


def action_lookup(index):
    """Helper function returns action given an index"""
    action_dic = {0:'u', 1:'d', 2:'l', 3:'r'}
    return action_dic[index]

def index_lookup(action):
    """Helper function returns index given action"""
    index_dic = {'u':0, 'd':1, 'l':2, 'r':3}
    return index_dic[action]


def next_state(state, action_index, neighbors = neighbors, action_lookup = action_lookup):
    return(neighbors[state][action_lookup[action_index]])

def simulate_environment(s, action, neighbors = neighbors, rewards = rewards, terminal = 0):
    """
    Function simulates the environment for Q-learning.
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward_prime = np.array([rewards[s_prime][a] for a in rewards[0].keys()])
    return (s_prime, reward_prime, is_terminal(s_prime, terminal))
    

def is_terminal(state, terminal = 0):
    return state == terminal

## Test the function
for a in ['u', 'd', 'r', 'l']:
    print(simulate_environment(1, a))
    

## Q-Learning

As you have already seen, the SARSA algorithm is an on policy action value TD estimation method. The **Q-learning** algorithm is a **off policy** TD action value estimation method. 

The update formula for single step Q-learning or **Q-learning(0)** is:

$$Q(S_t,A_t) = Q(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma\ max_a Q(S_{t+1},a) - Q(S_t,A_t) \big]$$  

Where,   
$\delta_t = R_{t+1} + \gamma max_a Q(S_{t+1},a) - Q(S_t,A_t) = $ the TD error,   
$max_a = $ the maximum operator applied to all possible actions in state $S_{t+1}$,   
$Q(S_t,A_t) = $ is the action value in state S given action A,  
$R_{t+1} = $ is the reward for the next time step,   
$\alpha = $ the learning rate,   
$\gamma = $ discount factor.  

The use of the operator $max_a$ makes Q-learning greedy. But, why does using this operator result in an off-policy algorithm? To answer this question, examine the backup diagram shown below. 

<img src="img/Q-Learning.JPG" alt="Drawing" style="width:200px; height:150px"/>
<center> **Backup Diagram for one-step Q-Learning** </center>

The $max_a$ greedily picks the action with the greatest value, regardless of policy. Therefore, Q-learning is an off-policy algorithm. 

### Q-Learning Example

The code in the cell below implements the one step Q-learning(0) algorithm. The code is nearly identical to the SARSA(0) code shown in Part 1 of this lab. The main difference is the addition of the $max_a$ operation when computing the TD error, $\delta_t$.  
 
The code in the cell below defines the initial random policy and a function to start an episode. Execute this code and test case and examine the results. 

In [ ]:
initial_policy = {0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}

def start_episode(n_states, n_actions):
    '''Function to find a random starting values for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state)):  ## Make sure not starting at the terminal state
         state = nr.choice(range(n_states))
    ## Now find a random starting action index
    a_index = nr.choice(range(4), size = 1)[0]
    s_prime, reward, terminal = simulate_environment(state, action_lookup(a_index))   
    return state, a_index, reward[a_index] ## action_lookup(a_index), reward[a_index]

## test the function to make sure never starting in terminal state
[start_episode(15,4) for _ in range(10)]

The function in the cell below finds and action given the state. The code executes two steps:

1. An action is found by random choice weighted by the probability of the action of the policy. 
2. Given the action the next state, reward and a terminal flag are found by calling the simulator function.     

Execute this code and examine the test case. 

In [ ]:
def take_action(state, policy):
    '''Function takes action given state using the transition probabilities 
    of the policy'''
    ## Find the action given the transistion probabilities defined by the policy.
    action = action_lookup(nr.choice(range(len(policy[0].keys())), p = list(policy[state].values()))) 
    s_prime, reward, terminal = simulate_environment(state, action)
    return (action, s_prime, reward, terminal)

## Test function for several states
for s in range(16):
    print(take_action(s, initial_policy))

The code in the cell below implements the Q-learning(0) algorithm to compute the action values for the Grid world given an policy. There are three functions:     

1. A helper function to neatly print the action values computed.     
2. A function to compute the update for the action-values by the following steps:
  - The next state, reward and terminal flag are returned by the `simulate_environment` function.
  - The successor action with the maximum action value is selected.
  - The action value is updated.
3. The last function execute the complete Q-learning algorithm. The main loop is over the number of episodes specified. The inner loop is over the steps of the episode and manges the bookkeeping required.     


### Exercise 1:

The code for the `Q_learning_0` function is incomplete. Add the missing line of code. Then execute the code to compute and print the action values for the random walk policy.     

In [ ]:
## Make a copy of the policy so as to not overwrite
import copy
temp_policy = copy.deepcopy(initial_policy)

def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)

def update_Q(Q, current_state, a_index, reward, alpha, gamma):
    """Function to update the actions values in the Q matrix"""
    ## Get s_prime given s and a
    s_prime, reward_prime, terminal = simulate_environment(current_state, action_lookup(a_index))
    a_prime_index = nr.choice(np.where(reward_prime == max(reward_prime))[0], size = 1)[0]
    ## Update the action values 
    
    ####### ADD MISSING CODE TO UPDATE Q ARRAY HERE  ##################
    Q[current_state,a_index] = Q[current_state,a_index] + alpha * (reward + gamma * (Q[s_prime,a_prime_index] - Q[current_state,a_index]))
    return Q, s_prime, reward_prime, terminal, a_prime_index

def Q_learning_0(policy, episodes, alpha = 0.2, gamma = 0.9):
    """
    Function to perform Q-learning(0) action-value estimation.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    
    ## Initialize Q matrix
    Q = np.zeros((n_states,n_actions))
    
    for _ in range(episodes): # Loop over the episodes
        terminal = False
        ## Find the inital state, action index and reward
        current_state, a_index, reward = start_episode(n_states,n_actions)
        
        while(not terminal): # Episode ends where get to terminal state   
            ## Update the action values in Q
            Q, s_prime, reward_prime, terminal, a_prime_index = update_Q(Q, current_state, a_index, reward, alpha, gamma)
            ## Set action, reward and state for next iteration
            a_index = a_prime_index
            current_state = s_prime
            reward = reward_prime[a_prime_index]
    return(Q)

nr.seed(9182)
Q = Q_learning_0(temp_policy, 1000)
print_Q(Q)

### Policy improvement

With the Q-learning algorithm available to compute Q-values, the policy can now be improved. Code to find an optimal policy using an $/epsilon$-greedy algorithm is in the cell below. The function accounts for the fact that there may be more than one action which results in a maximum action value. Additional details can be found n the code comments.  

Execute this code and examine the results.  

In [ ]:
def update_policy(policy, Q, epsilon):
    '''Updates the policy based on estiamtes of Q using 
    an epslion greedy algorithm. The action with the highest
    action value is used.'''
    
    ## Find the keys for the actions in the policy
    keys = list(policy[0].keys())
    
    ## Iterate over the states and find the maximm action value.
    for state in range(len(policy)):
        ## First find the index of the max Q values  
        q = Q[state,:]
        max_action_index = np.where(q == max(q))[0]

        ## Find the probabilities for the transitions
        n_transitions = float(len(q))
        n_max_transitions = float(len(max_action_index))
        p_max_transitions = (1.0 - epsilon *(n_transitions - n_max_transitions))/(n_max_transitions)
  
        ## Now assign the probabilities to the policy as epsilon greedy.
        for key in keys:
            if(index_lookup(key) in max_action_index): policy[state][key] = p_max_transitions
            else: policy[state][key] = epsilon
    return(policy)                

improved_policy_1 = update_policy(initial_policy, Q, 0.01)    
improved_policy_1

### Double Q-Learning

The Q-learning algorithm just presented has a significant **bias**. To understand why this might be consider the following thought experiment. In most cases the sampled action values are inaccurate. Some action values will have a positive error and some will have a negative error. In the error is on the order of the values themselves, the $max_a$ operator has a reasonable chance of selecting an action value that is the largest because of this error. However, the $max_a$ operator will never select an action with a low value simply because of the errors. The net result is a bias toward action values with the largest positive error. 

What can be done to correct this situation? One relatively simple and effective algorithm is known as **double Q-learning**. Double Q-learning maintains two tables of action values. The values from one table are used to perform the bootstrap updates of the other table and vice versa. This approach averages out the bias. For two tables, $Q_1$ and $Q_2$ we can express double Q-learning as follows:

$$Q_1(S_t,A_t) = Q_1(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_2 \big( S_{t+1},argmax_a Q_1(S_{t+1} , a) \big) - Q_1(S_t,A_t) \big] \\
Q_2(S_t,A_t) = Q_2(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_1 \big( S_{t+1},argmax_a Q_2(S_{t+1} , a) \big) - Q_2(S_t,A_t) \big]$$  

With a 0.5 probability one or the other of these expressions is used for the TD update at each time step. While double Q-learning requires twice as much memory, to maintain the two tables, the computational complexity is the same when compared to Q-learning. 

> **Note:** Another **unbiased** one step off policy TD algorithm is known as **expected SARSA**. See Section 6.6 of Sutton and Barto, second edition, for details.   

### Example of Double Q-Learning

The code in the cell below implements the double Q-learning algorithm. The steps are essentially the same as the foregoing code, except for the updates of the two Q tables. Additional details on this algorithm can be seen by reading the code comments.  

### Exercise 2:

The `double_Q_learning_0` function is missing a critical piece. The code to update the two action-value arrays, Q1 and Q2. Add the code to update these arrays randomly with a probability of 0.5 that a given array is updated. The execute the code and examine the results. 

In [ ]:
## Make a copy of the policy so as to not overwrite
temp_policy = copy.deepcopy(initial_policy)

def update_double_Q(q1, q2, current_state, a_index, reward, alpha, gamma):
    """Function to update the actions values in the Q matrix"""
    ## Get s_prime given s and a
    s_prime, reward_prime, terminal = simulate_environment(current_state, action_lookup(a_index))
    a_prime_index = nr.choice(np.where(reward_prime == max(reward_prime))[0], size = 1)[0]
    ## Update the action values 
    q1[current_state,a_index] = q1[current_state,a_index] + alpha * (reward + gamma * (q2[s_prime,a_prime_index] - q1[current_state,a_index]))
    return q1, s_prime, reward_prime, terminal, a_prime_index


def double_Q_learning_0(policy, episodes, alpha = 0.2, gamma = 0.9):
    """
    Function to perform Q-learning(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    
    ## Initialize both Q matricies
    Q1 = np.zeros((n_states,n_actions))
    Q2 = np.zeros((n_states,n_actions))
    
    for _ in range(episodes): # Loop over the episodes
        terminal = False
        ## Find the inital state, action index and reward
        current_state, a_index, reward = start_episode(n_states,n_actions)
        
        while(not terminal): # Episode ends where get to terminal state   
            ## Update the action values in Q1 or Q2 based on random choice
            
            ####### ADD THE MISSING CODE TO RANDOMLY UPDATE Q1 or Q2 WITH   #######
            ####### PROBABIITY OF 0.5 TO UPDATE EITHER ACION VALUE ARRAY.   #######
            ####### MAKE USE OF CALLS TO THE update_double_Q FUNCTION.      #######
            if(nr.uniform() <= 0.5):
                Q1, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q1, Q2, current_state, a_index, reward, alpha, gamma)
            else:
                Q2, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q2, Q1, current_state, a_index, reward, alpha, gamma)
            ## Set action, reward and state for next iteration
            a_index = a_prime_index
            current_state = s_prime
            reward = reward_prime[a_prime_index]
    return(Q1)

nr.seed(4123)
double_Q = double_Q_learning_0(temp_policy, 1000)
print_Q(double_Q)

In [ ]:
improved_policy_2 = update_policy(initial_policy, double_Q, 0.01)   
improved_policy_2